In [1]:
import os
from dotenv import load_dotenv

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [5]:
# Sample Football player documents
docs = [
    Document(
        page_content="Lionel Messi is considered one of the greatest footballers of all time. Known for his incredible dribbling, vision, and goal-scoring ability, he has won multiple Ballon d'Or awards. He currently plays for Inter Miami after a legendary career at Barcelona.",
        metadata={"club": "Inter Miami"}
    ),
    Document(
        page_content="Cristiano Ronaldo is one of the most iconic forwards in football history. Famous for his athleticism, powerful shots, and leadership, he has won major trophies across England, Spain, and Italy. He now plays for Al Nassr in the Saudi Pro League.",
        metadata={"club": "Al Nassr"}
    ),
    Document(
        page_content="Kylian Mbappé is a French forward known for his explosive pace and clinical finishing. A World Cup winner at just 19, he has become the face of Paris Saint-Germain and is regarded as one of the brightest talents in modern football.",
        metadata={"club": "Paris Saint-Germain"}
    ),
    Document(
        page_content="Kevin De Bruyne is a world-class midfielder playing for Manchester City. Renowned for his passing accuracy, vision, and ability to control the tempo of a game, he has been a key figure in City's domestic and European success.",
        metadata={"club": "Manchester City"}
    ),
    Document(
        page_content="Virgil van Dijk is one of the top defenders in the world, commanding Liverpool’s backline with authority. His strength, aerial ability, and calmness under pressure have transformed Liverpool's defense in recent years.",
        metadata={"club": "Liverpool"}
    ),
]

In [6]:
# Initialize Google Gemini Embeddings
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # default Gemini embedding model
    task_type="retrieval_document" # can also use 'retrieval_query' for query vectors
)

In [7]:
# Create or load Chroma vector store
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='my_chroma_db_gemini',
    collection_name='sample_gemini'
)

C:\Users\emon1\AppData\Local\Temp\ipykernel_7420\1129830283.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [8]:
# Add documents
vector_store.add_documents(docs)

['6c5d3258-d3cd-4fa4-ab77-8ad6bbd4501c',
 'b62a6916-f1da-47cd-b76c-ad0f94728d6c',
 'a7e8361a-4ba3-48e5-a1d4-43c763fc44a9',
 'ae206513-1c08-4fef-990b-4d786bd1a45f',
 '4aaf37bf-e0ab-4214-a94e-23ab8c173a9f']

In [9]:
# View all documents
print("\n--- All Documents ---")
print(vector_store.get(include=['documents', 'metadatas']))


--- All Documents ---
{'ids': ['6c5d3258-d3cd-4fa4-ab77-8ad6bbd4501c', 'b62a6916-f1da-47cd-b76c-ad0f94728d6c', 'a7e8361a-4ba3-48e5-a1d4-43c763fc44a9', 'ae206513-1c08-4fef-990b-4d786bd1a45f', '4aaf37bf-e0ab-4214-a94e-23ab8c173a9f'], 'embeddings': None, 'documents': ["Lionel Messi is considered one of the greatest footballers of all time. Known for his incredible dribbling, vision, and goal-scoring ability, he has won multiple Ballon d'Or awards. He currently plays for Inter Miami after a legendary career at Barcelona.", 'Cristiano Ronaldo is one of the most iconic forwards in football history. Famous for his athleticism, powerful shots, and leadership, he has won major trophies across England, Spain, and Italy. He now plays for Al Nassr in the Saudi Pro League.', 'Kylian Mbappé is a French forward known for his explosive pace and clinical finishing. A World Cup winner at just 19, he has become the face of Paris Saint-Germain and is regarded as one of the brightest talents in modern foo

In [12]:
# Similarity search (no score)
print("\n--- Similarity Search: Greatest ---")
results = vector_store.similarity_search(
    query='Who among these are a greatest?',
    k=2
)
for doc in results:
    print(doc.page_content)


--- Similarity Search: Greatest ---
Lionel Messi is considered one of the greatest footballers of all time. Known for his incredible dribbling, vision, and goal-scoring ability, he has won multiple Ballon d'Or awards. He currently plays for Inter Miami after a legendary career at Barcelona.
Virgil van Dijk is one of the top defenders in the world, commanding Liverpool’s backline with authority. His strength, aerial ability, and calmness under pressure have transformed Liverpool's defense in recent years.


In [13]:
# Similarity search with scores
print("\n--- Similarity Search with Score: Defender ---")
scored_results = vector_store.similarity_search_with_score(
    query='Who among these are a defender?',
    k=2
)
for doc, score in scored_results:
    print(f"Score: {score:.4f} | Content: {doc.page_content}")


--- Similarity Search with Score: Defender ---
Score: 0.5208 | Content: Virgil van Dijk is one of the top defenders in the world, commanding Liverpool’s backline with authority. His strength, aerial ability, and calmness under pressure have transformed Liverpool's defense in recent years.
Score: 0.6188 | Content: Lionel Messi is considered one of the greatest footballers of all time. Known for his incredible dribbling, vision, and goal-scoring ability, he has won multiple Ballon d'Or awards. He currently plays for Inter Miami after a legendary career at Barcelona.


In [22]:
print("\n--- Filter by Team ---")
filtered_results = vector_store.similarity_search_with_score(
    query='',
    filter={"team": "Paris Saint-Germain"}
)
for doc, score in filtered_results:
    print(f"Score: {score:.4f} | Content: {doc.page_content}")


--- Filter by Team ---
